### NOTEBOOK for constructiong POD-RBF NIROM approximation for a shallow water example

A collection of high-fidelity snapshots are generated that sufficiently capture the time-dynamics of the simulation. POD is adopted to define a reduced basis space for the high-fidelity snaphosts. RBF interpolation is adopted to approximate the evolution of the time dynamics in the reduced space spanned by the POD modes.  



In [ ]:
## Load modules
import numpy as np
import scipy
from importlib import reload

import os
import gc
from importlib import reload
from scipy import interpolate
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, ScalarFormatter, FormatStrFormatter

from matplotlib import animation
matplotlib.rc('animation', html='html5')
from IPython.display import display
import matplotlib.ticker as ticker
from matplotlib import rcParams
from matplotlib.offsetbox import AnchoredText

# Plot parameters
plt.rc('font', family='serif')
plt.rcParams.update({'font.size': 20,
                     'lines.linewidth': 2,
                     'axes.labelsize': 16, # fontsize for x and y labels (was 10)
                     'axes.titlesize': 20,
                     'xtick.labelsize': 16,
                     'ytick.labelsize': 16,
                     'legend.fontsize': 16,
                     'axes.linewidth': 2})

import itertools
colors = itertools.cycle(['r','g','b','m','y','c'])
markers = itertools.cycle(['p','d','o','^','s','x','D','H','v','*'])

base_dir = os.getcwd()
src_dir  = os.path.join(base_dir,'../src/podrbf/')
work_dir = os.path.join(base_dir,'../notebooks/')
data_dir = os.path.join(base_dir,'../data/')
nirom_dir = os.path.join(base_dir,'../data/')
fig_dir  = os.path.join(base_dir,'../figures/rbf/')

os.chdir(src_dir)
import main as prb
import plotting as pu
import rbf as rbf
import pod as pod
os.chdir(work_dir)


In [ ]:
## Load snapshot data

# ### San Diego problem
# model = 'SD'
# data = np.load(os.path.join(data_dir,'san_diego_tide_snapshots_T4.32e5_nn6311_dt25.npz'))
# mesh = np.load(os.path.join(data_dir,'san_diego_mesh.npz'))

### Red River problem
model ='Red'
data = np.load(os.path.join(data_dir,'red_river_inset_snapshots_T7.0e4_nn12291_dt10.npz'))
mesh = np.load(os.path.join(data_dir,'red_river_mesh.npz'))

print("Solution component keys are : " + str(list(data.keys())))
print("Mesh element keys are : " + str(list(mesh.keys())))

In [ ]:
## Prepare training snapshots
soln_names = ['S_dep', 'S_vx', 'S_vy']
comp_names={0:'S_dep',1:'S_vx',2:'S_vy'}
Nc=3 

nodes = mesh['nodes']; triangles = mesh['triangles']
Nn = nodes.shape[0]; Ne = triangles.shape[0]


snap_start = 100
if model == 'SD':
    T_end = 50*3600   ### 50 hours in seconds
elif model == 'Red':
    T_end = 3.24e4    ### 9 hours in seconds
snap_end = np.count_nonzero(data['T'][data['T'] <= T_end])

snap_data = {}
for key in soln_names:
    snap_data[key] = data[key][:,snap_start:]

times_offline = data['T'][snap_start:]
Nt = times_offline.size
print('Loaded {0} snapshots of dimension {1} for h,u and v, spanning times [{2}, {3}]'.format(
                    snap_data[soln_names[0]].shape[1],snap_data[soln_names[0]].shape[0], 
                    times_offline[0], times_offline[-1]))


## number of steps to skip in selecting training snapshots for SVD basis
if model == 'SD':
    snap_incr=4
elif model == 'Red':
    snap_incr=3
## Subsample snapshots for building POD basis
snap_train = {};
for key in soln_names:
    snap_train[key] = snap_data[key][:,:snap_end+1:snap_incr]


times_train=times_offline[:snap_end+1:snap_incr]
Nt_b = times_train.size
print('Using {0} training snapshots for time interval [{1:.2f},{2:.2f}] hrs'.format(times_train.shape[0], 
                                        times_train[0]/3600, times_train[-1]/3600))


del data
del mesh
gc.collect()

In [ ]:
## Instantiate the POD-RBF class
os.chdir(src_dir)
reload(prb)

if model == 'SD':
#     trunc_lvl = 0.999995  #Used for GPR
    trunc_lvl = 0.9999995
    eps = 0.01
    
elif model == 'Red':
#     trunc_lvl = 0.9999995   #Used for GPR 
    trunc_lvl = 0.99
    eps = 0.05
    
PRB = prb.PODRBFBase(trunc = trunc_lvl)

In [ ]:
## Compute the POD basis for the space of snapshots
Phi, Sigma, Z_train = PRB.compute_pod_basis(snap_train, times_train)
dzdt_train = PRB._dzdt_train

## Compute the RBF approximation of the time derivative of
## POD modal coefficients

S_rbf = {}
if model == 'Red':
    rbf_incr = snap_incr+2
elif model == 'SD':
    rbf_incr = snap_incr-1

for key in soln_names:
    S_rbf[key] = snap_data[key][:,:snap_end+1:rbf_incr]
times_rbf = times_offline[:snap_end+1:rbf_incr]
Z_rbf = pod.project_onto_basis(S_rbf,PRB.basis, PRB._S_mean)
A_rbf, rbf_centers_rbf, rbf_coeff_rbf = PRB.fit_rbf(Z_rbf, times_rbf, kernel='matern',eps=eps)



In [ ]:
## Set the time steps for online prediction

t0 = times_train[0]
if model == 'Red':
    Tonline_end = 3.24e4  ### 9 hours in seconds
elif model == 'SD':
    Tonline_end = 50*3600   ### 50 hours in seconds
trainT0 = np.searchsorted(times_offline, t0); 
trainT = np.searchsorted(times_offline, times_train[-1]); 
trainP = np.searchsorted(times_offline, Tonline_end); 

finer_steps = True
long_term = True

if finer_steps and not long_term:
    onl_incr = snap_incr-1
    times_online = times_offline[trainT0:trainT+1:onl_incr]
    N_online = trainT+1
elif long_term and not finer_steps:
    onl_incr = snap_incr
    times_online = times_offline[trainT0:trainP+1:onl_incr]
    N_online = trainP+1
elif long_term and finer_steps:
    onl_incr = snap_incr-2
    times_online = times_offline[trainT0:trainP+1:onl_incr]
    N_online = trainP+1
Nt_online = times_online.size
print('Trying to simulate interval [{0:.2f},{1:.2f}] days with {2} steps'.format(t0,
                                                times_online[-1], Nt_online))


In [ ]:
## Evaluate NIROM online at queried time points
# reload(prb)

## RBF NIROM solution
uh, zh = PRB.predict_time(times_online, use_greedy=False)



### Visualize domain and ROM results

In [ ]:
## --- Visualize the singular values
# os.chdir(src_dir)
# reload(pu)
pu.plot_sing_val(Sigma)
plt.ylabel('$\ln{|\sigma|}$',fontsize=16)
plt.xlabel('Modes', fontsize=16)

# os.chdir(fig_dir)
# plt.savefig('%s_sing_value.png'%model,dpi=300,bbox_inches='tight')

In [ ]:
## --- Visualize l2-norm of time derivative of modal coefficients
dZdata_norm = {}
for ii,key in enumerate(soln_names):
    dZdata_norm[key] = np.linalg.norm(dzdt_train[key],axis=0);

fig = plt.figure(figsize=(12,3))
ky = soln_names[1]; ky1 = soln_names[2]; ky2 = soln_names[0]
t_indx = times_train[:-1]/3600
plt.plot(t_indx[:],(dZdata_norm[ky][:]),'r-o', markersize=8,markevery=100,label='$u_x$',linewidth=2)
plt.plot(t_indx[:],(dZdata_norm[ky1][:]),'g-D', markersize=8,markevery=100,label='$u_y$',linewidth=2)
plt.plot(t_indx[:],(dZdata_norm[ky2][:]),'b-^', markersize=8,markevery=100,label='$p$',linewidth=2)

ax = plt.gca()
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
lg = plt.legend(fontsize=16,ncol=3,loc='upper right') 
plt.xlabel('Time (hours)',fontsize=16)

# os.chdir(fig_dir)
# plt.savefig('%s_nirom_mode_norms.pdf'%model, bbox_inches='tight')


In [ ]:
## --- Compare the HFM and NIROM solution
# os.chdir(src_dir)
# reload(pu)

def set_label(key):
    if key == 'S_vx':
        ky = 'u'
    elif key == 'S_vy':
        ky = 'v'
    elif key == 'S_dep':
        ky = 'h'
    return ky

key = 'S_vx'; iplot = 1200
iplot_true = np.argmin(np.abs(times_offline-times_online[iplot]))

fig = plt.figure(figsize=(16,6))
urom = uh[key][:,iplot]
utrue = snap_data[key][:,iplot_true]
print("comparing NIROM solution at t = {1:.2f} hrs and fine-grid solution at t = {0:.2f} hrs".format(
                                            times_offline[iplot_true]/3600, times_online[iplot]/3600))

ax1 = plt.subplot(1,2,1); ax1.axis('off')
surf1 = ax1.tripcolor(nodes[:,0], nodes[:,1],triangles, urom, cmap=plt.cm.jet)
# cf1, boundaries_interp = pu.viz_sol(urom,nodes,triangles)
ax1.set_title("NIROM solution\n $%1.5f<\mathbf{%s}<%1.5f$"%(np.amin(urom),set_label(key),
                                                             np.amax(urom)),fontsize=16)
plt.axis('off')
plt.colorbar(surf1, shrink=0.8,aspect=20, pad = 0.03)


ax2 = plt.subplot(1,2,2); ax2.axis('off')
surf2 = ax2.tripcolor(nodes[:,0], nodes[:,1],triangles, utrue, cmap=plt.cm.jet)
# cf2, boundaries_true = pu.viz_sol(utrue,nodes,triangles)
ax2.set_title("HFM solution\n $%1.5f<\mathbf{%s}<%1.5f$"%(np.amin(utrue),set_label(key),
                                                             np.amax(utrue)),fontsize=16)
plt.axis('off')
plt.colorbar(surf2, shrink=0.8,aspect=20, pad = 0.03)



In [ ]:
## Visualize NIROM solution

print("NIROM solution at t = {0:.2f} hrs".format(times_online[iplot]/3600))
tn = times_online[iplot]
fig = plt.figure(figsize=(8,6))
ax1 = plt.subplot(1,1,1); ax1.axis('off')
surf1 = ax1.tripcolor(nodes[:,0], nodes[:,1],triangles, urom, cmap=plt.cm.jet)
# cf1, boundaries_interp = pu.viz_sol(urom,nodes,triangles)
ax1.set_title("RBF solution at t=%.2f hrs\n $%1.5f<\mathbf{%s}<%1.5f$"%(tn/3600,np.amin(urom),set_label(key),
                                                             np.amax(urom)),fontsize=16)
plt.axis('off')
plt.colorbar(surf1, shrink=0.8,aspect=20, pad = 0.03)

# os.chdir(fig_dir)
# plt.savefig('%s_rbf_%s_t%.3f_tskip%d_oskip%d.pdf'%(model,set_label(key),times_online[iplot]/3600,snap_incr,onl_incr),bbox_inches='tight')

In [ ]:
## Visualize NIROM error
print("comparing NIROM solution at t = {1:.2f} hrs and fine-grid solution at t = {0:.2f} hrs".format(
                                        times_offline[iplot_true]/3600, times_online[iplot]/3600))

fig = plt.figure(figsize=(8,6))
ax3 = plt.subplot(1,1,1); ax1.axis('off')
surf3 = ax3.tripcolor(nodes[:,0], nodes[:,1],triangles, utrue-urom, cmap=plt.cm.jet)

boundaries_err = [np.amin(utrue-urom), np.amax(utrue-urom)]
ax3.set_title("$%1.6f <$ RBF Error $< %1.6f$\n Rel. Error 2-norm : %2.6f"%(boundaries_err[0],
                boundaries_err[1],np.linalg.norm(urom-utrue)/np.linalg.norm(utrue)),fontsize=16)
plt.axis('off')
plt.colorbar(surf3, shrink=0.8,aspect=20, pad = 0.03)

# os.chdir(fig_dir)
# plt.savefig('%s_rbf_relerr_%s_t%.3f_tskip%d_oskip%d.pdf'%(model,set_label(key),times_online[iplot]/3600,snap_incr,onl_incr), bbox_inches='tight')

In [ ]:
### Compute spatial RMS errors

fig = plt.figure(figsize=(16,4))
start_trunc = 10+0*np.searchsorted(times_online,times_train[-1])//10
end_trunc = 10*np.searchsorted(times_online,times_train[-1])//10
end_trunc = end_trunc + (Nt_online - end_trunc)//1
x_inx = times_online/3600
t_ind = np.searchsorted(times_offline, times_online)

ky1 = 'S_dep'; ky2 = 'S_vx'; ky3 = 'S_vy'
md1 = set_label(ky1); md2 = set_label(ky2); md3 = set_label(ky3)

rms_err = {}
fig = plt.figure(figsize=(16,4))
for ivar,key in enumerate(soln_names):
    rms_err[key] = np.linalg.norm(uh[key][:,:] - snap_data[key][:,t_ind], axis = 0)/np.sqrt(Nn)  #\
#                             np.linalg.norm(snap_data[key][:,t_ind], axis = 0)


ax1 = fig.add_subplot(1, 2, 1)
ax1.plot(x_inx[start_trunc:end_trunc], rms_err[ky1][start_trunc:end_trunc], 'r-s', markersize=8,
                label='$\mathbf{%s}$'%(md1),lw=2,markevery=500)
ax1.set_xlabel('Time (hours)');lg=plt.legend(ncol=2,fancybox=True,)


ax2 = fig.add_subplot(1, 2, 2)
ax2.plot(x_inx[start_trunc:end_trunc], rms_err[ky2][start_trunc:end_trunc], 'b-o', markersize=8,
                label='$\mathbf{%s}$'%(md2), lw=2,markevery=400)
ax2.plot(x_inx[start_trunc:end_trunc], rms_err[ky3][start_trunc:end_trunc], 'g-^', markersize=8,
                label='$\mathbf{%s}$'%(md3), lw=2,markevery=460)
ax2.set_xlabel('Time (hours)');lg=plt.legend(ncol=2,fancybox=True,)
 
# os.chdir(fig_dir)
# plt.savefig('%s_rbf_rms_tskip%d_oskip%d.pdf'%(model,snap_incr,onl_incr),bbox_extra_artists=(lg,), bbox_inches='tight')

In [ ]:
## Save the NIROM solutions to disk
os.chdir(nirom_dir)
np.savez_compressed('%s_online_rbf'%model,S_dep=uh['S_dep'],S_vx = uh['S_vx'], S_vy = uh['S_vy'],
                                                    zS_dep=zh['S_dep'],zS_vx = zh['S_vx'], zS_vy = zh['S_vy'],
                                                    time=times_online)

os.chdir(work_dir)

In [ ]:
## Saving the ROM model
# os.chdir(data_dir)
# filename='podrbf_rom_sandiego'
# filename='podrbf_rom_redriver'
# PRB.save_to_disk(filename,PRB)